In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
# exmine whether data fileds exits
my_data_path = '/dbstore/UKBiobank/Data_Download_02JULY2020/ukb42608.csv'
my_data = pd.read_csv(my_data_path,nrows=5)
total_items = my_data.columns.values

depression_items = ['eid']
anxiety_items = ['eid']
control_items = ['eid']
for item in depression_fields:
    l = len(item)
    t = []
    for c in total_items:
        if c[:l+1] == item + '-':
            t.append(c)
            depression_items.append(c)
    if len(t) == 0:
        print(item,'not exits')

for item in anxiety_fields:
    l = len(item)
    t = []
    for c in total_items:
        if c[:l+1] == item + '-':
            t.append(c)
            anxiety_items.append(c)
    if len(t) == 0:
        print(item,'not exits')

for item in control_fields:
    l = len(item)
    t = []
    for c in total_items:
        if c[:l+1] == item + '-':
            t.append(c)
            control_items.append(c)
    if len(t) == 0:
        print(item,'not exits')

In [11]:
# load CSA daa
area_items = pd.read_csv('/data/sliu/imaging_genetics/DK_structure/Area_items.csv')
s1 = ['eid']
for i in range(area_items.shape[0]):
    s1.append(str(area_items.iloc[i,0])+'-2.0')
data1 = pd.read_csv('/data/sliu/UKBB_phenotypes/ukb42608.csv',usecols=s1)
data1.dropna(axis=0,how='any',inplace=True)
# load CT data
thickness_items = pd.read_csv('/data/sliu/imaging_genetics/DK_structure/Thickness_items.csv')
s2= ['eid']
for i in range(thickness_items.shape[0]):
    s2.append(str(thickness_items.iloc[i,0])+'-2.0')
data2 = pd.read_csv('/data/sliu/UKBB_phenotypes/ukb42608.csv',usecols=s2)
data2.dropna(axis=0,how='any',inplace=True)

In [14]:
# select participants with both structural and functional imaging data
import os
files = os.listdir('/dbstore/UKBiobank/RSFMRI')
rsfmri_list = []
for f in files:
    if f[-7:] == '2_0.zip':
        file_path = os.path.join('/dbstore/UKBiobank/RSFMRI',f)
        size2 = os.path.getsize(file_path)
        if size2 > 500000654:
            rsfmri_list.append(int(f[:7]))

smri_list = list(set(data1['eid'].values))
subjs = list(set(rsfmri_list) & set(smri_list))
# subjs = list(data1['eid'].values)

In [17]:
# select rsfMRI head motion, total gray matter volume, ICV, total cortical surface area, average cortical thickness
cols = ['eid','25741-2.0','25005-2.0','26521-2.0','26721-2.0','26822-2.0','26755-2.0','26856-2.0']

cov1 = pd.read_csv('/data/sliu/UKBB_phenotypes/ukb42608.csv',usecols=cols)

In [18]:
# select sex, age, IQ, EA from Karin's ukbb data and matched IDs with Guido's ukbb data
cols2 = ['eid','31-0.0','21003-2.0','20016-2.0','6138-2.0']
cov2 = pd.read_csv('/data/sliu/UKBB_phenotypes/ukb30545.csv',usecols=cols2)
cov2.set_index('eid',inplace=True)
matched_IDs = pd.read_csv('/data/sliu/updated_ukbb2/matched_IDs2.csv')
matched_IDs.set_index('guido_id',inplace=True)
ll = list(set(matched_IDs.index) & set(data2['eid'].values))
temp_l = matched_IDs.loc[ll]['karin_id']
info_pheno = cov2.loc[temp_l]
info_pheno.reset_index(inplace=True)
info_pheno['guido_id'] = ll
info_pheno.dropna(how='any',inplace=True)
info_pheno.rename(columns={'eid':'karin_id','guido_id':'eid'},inplace=True)

In [25]:
# transfer EA
# tdata = pd.merge(cov1,imaging_data,on='eid',how='inner')
tdata = pd.merge(cov1,info_pheno,on='eid',how='inner')
tdata.set_index('eid',inplace=True)
dl = []
for i in range(tdata.shape[0]):
    if tdata['6138-2.0'].iloc[i] == -3:
        dl.append(tdata.index[i])
    if tdata['6138-2.0'].iloc[i] == 1:
        tdata['6138-2.0'].iloc[i] = 20
    if tdata['6138-2.0'].iloc[i] == 2:
        tdata['6138-2.0'].iloc[i] = 13
    if tdata['6138-2.0'].iloc[i] == 3:
        tdata['6138-2.0'].iloc[i] = 10
    if tdata['6138-2.0'].iloc[i] == 4:
        tdata['6138-2.0'].iloc[i] = 10
    if tdata['6138-2.0'].iloc[i] == 5:
        tdata['6138-2.0'].iloc[i] = 19 
    if tdata['6138-2.0'].iloc[i] == 6:
        tdata['6138-2.0'].iloc[i] = 15
    if tdata['6138-2.0'].iloc[i] == -7:
        tdata['6138-2.0'].iloc[i] = 7   
tdata.drop(dl,inplace=True)

/data/sliu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [26]:
tdata.dropna(how='any',inplace=True)
subjs1 = list(set(subjs) & set(tdata.index))

In [28]:
# current depression or anxiety status while scanning
my_data_path = '/dbstore/UKBiobank/Data_Download_02JULY2020/ukb42608.csv'
current_status_fields = ['eid','2050-2.0','2060-2.0','2070-2.0']
current_status_data = pd.read_csv(my_data_path,usecols=current_status_fields)
current_status_data.set_index('eid',inplace=True)
current_status_data2 = current_status_data.loc[subjs1]
current_status_data2.reset_index(inplace=True)
current_status_data2.dropna(how='any',inplace=True)
current_status_data2.shape

(20754, 4)

In [29]:
# select individuals with depression or anxiety 
l1 = []
l2 = []
for i in range(current_status_data2.shape[0]):
    if current_status_data2['2050-2.0'].iloc[i] >= 3 or current_status_data2['2060-2.0'].iloc[i] >= 3:
        l1.append(current_status_data2['eid'].iloc[i])
    if current_status_data2['2070-2.0'].iloc[i] >= 3:
        l2.append(current_status_data2['eid'].iloc[i])

In [30]:
# overlappping betwenn individuals with depression and those with anxiety 
common_subjs = list(set(l1) & set(l2))
pure_dp_subjs = list(set(l1) - set(common_subjs))
pure_gad_subjs = list(set(l2) - set(common_subjs))

print("number of subjects with both current depression and anxiety: ", len(common_subjs))
print("number of subjects only with current depression: ", len(pure_dp_subjs))
print("number of subjects only with current anxiety: ", len(pure_gad_subjs))

number of subjects with both current depression and anxiety:  276
number of subjects only with current depression:  532
number of subjects only with current anxiety:  235


In [60]:
# define controls
control_fields = ['20544','20002','20514','20510','20517','20519','20511','20507','20508','20518','20513',\
                 '20506','20509','20520','20515','20516','20505','20512']

history_depression_fields = ['20446','20441','20436','20439','20440','20446','20441','20449','20536','20532',\
                     '20435','20450','20437']
history_anxiety_fields = ['20421','20420','20538','20425','20542','20543','20540','20541','20539','20537','20418',\
                  '20426','20423','20429','20419','20422','20417','20427']

In [61]:
# exmine whether data fileds exits
my_data_path = '/dbstore/UKBiobank/Data_Download_02JULY2020/ukb42608.csv'
my_data = pd.read_csv(my_data_path,nrows=5)
total_items = my_data.columns.values

history_depression_items = ['eid']
history_anxiety_items = ['eid']
control_items = ['eid']
for item in history_depression_fields:
    l = len(item)
    t = []
    for c in total_items:
        if c[:l+1] == item + '-':
            t.append(c)
            history_depression_items.append(c)
    if len(t) == 0:
        print(item,'not exits')

for item in history_anxiety_fields:
    l = len(item)
    t = []
    for c in total_items:
        if c[:l+1] == item + '-':
            t.append(c)
            history_anxiety_items.append(c)
    if len(t) == 0:
        print(item,'not exits')

for item in control_fields:
    l = len(item)
    t = []
    for c in total_items:
        if c[:l+1] == item + '-':
            t.append(c)
            control_items.append(c)
    if len(t) == 0:
        print(item,'not exits')

In [62]:
# select participants who attend the scannings
scanning_fields = ['eid','27143-2.0']
present_scanning_data = pd.read_csv(my_data_path,usecols=scanning_fields).dropna(how='any')
present_scanning_subjs = present_scanning_data.iloc[:,0].values

DP_data = pd.read_csv(my_data_path,usecols=depression_items)
DP_data.set_index('eid',inplace=True)
DP_data2 = DP_data.loc[present_scanning_subjs]
DP_data2.reset_index(inplace=True)

GAD_data = pd.read_csv(my_data_path,usecols=anxiety_items)
GAD_data.set_index('eid',inplace=True)
GAD_data2 = GAD_data.loc[present_scanning_subjs]
GAD_data2.reset_index(inplace=True)

HC_data = pd.read_csv(my_data_path,usecols=control_items)
HC_data.set_index('eid',inplace=True)
HC_data2 = HC_data.loc[present_scanning_subjs]
HC_data2.reset_index(inplace=True)

In [63]:
# select data fields to define depression ever and controls

# Self-reported depression (SRD) in field 20002.2.x

# gather all the arrays in 20002.2.x
srd = HC_data2.filter(like='20002-2')

mental_health_code = [1286,1287,1288,1289,1290,1291,1469,1470,1614,1615,1616]

for i in range(len(mental_health_code)):
    code = mental_health_code[i]
    print(code)
    added_item = 'Self_Reported_'+ str(code) + '_Imaging_Time'
    HC_data2[added_item] = srd.apply(lambda row: (row == code).any(), axis=1).replace((1,0),('Yes','No'))
    
HC_data2['Self_Reported_Mental_Health_Imaging_Time'] = np.sum(HC_data2.iloc[:,-11:].values == 'Yes',axis=1)

mhp = HC_data2.filter(like='20544-0')

# Ever_Diagnosed_Depression -> used for building Lifetime and Remitted Depression
HC_data2['Ever_Diagnosed_Mental_Health_Problem'] = mhp.apply(lambda row: ~row.isnull().all(), axis=1)\
                                             .replace((1,0),('Yes','No'))

1286
1287
1288
1289
1290
1291
1469
1470
1614
1615
1616


In [64]:
new_control_items = []
for c in control_items:
    if c[:6] != '20544-' and c[:6] != '20002-':
        new_control_items.append(c)
new_control_items.append('Self_Reported_Mental_Health_Imaging_Time')
new_control_items.append('Ever_Diagnosed_Mental_Health_Problem')
HC_data2 = HC_data2[new_control_items]

In [65]:
# Define history depressed subjects
history_dp_subjs = []
for i in range(DP_data2.shape[0]):
    num_criterias = 0
    if DP_data2['20446-0.0'].iloc[i] == 1 or DP_data2['20441-0.0'].iloc[i] == 1:
        num_criterias = num_criterias + 1
    if DP_data2['20436-0.0'].iloc[i] >= 3:
        num_criterias = num_criterias + 1
    if DP_data2['20439-0.0'].iloc[i] >= 2:
        num_criterias = num_criterias + 1
    if DP_data2['20440-0.0'].iloc[i] >= 2:
        num_criterias = num_criterias + 1 
    
    # Total number of symptoms endorsed (core and others) >= 5
    num_symptoms = 0
    if DP_data2['20446-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if DP_data2['20441-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1 
    if DP_data2['20449-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if DP_data2['20536-0.0'].iloc[i] > 0:
        num_symptoms = num_symptoms + 1
    if DP_data2['20532-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1 
    if DP_data2['20435-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if DP_data2['20450-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if DP_data2['20437-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1        
    
    if num_symptoms >= 5:
        num_criterias = num_criterias + 1
        
    if num_criterias == 5:
        history_dp_subjs.append(DP_data2['eid'].iloc[i])

In [66]:
# Define history anxiety subjects
history_gad_subjs = []
for i in range(GAD_data2.shape[0]):
    num_criterias = 0
    if GAD_data2['20421-0.0'].iloc[i] == 1:
        num_criterias = num_criterias + 1
    if GAD_data2['20420-0.0'].iloc[i] >= 6 or GAD_data2['20420-0.0'].iloc[i] == -999:
        num_criterias = num_criterias + 1
    if GAD_data2['20538-0.0'].iloc[i] == 1:
        num_criterias = num_criterias + 1
    if GAD_data2['20425-0.0'].iloc[i] == 1 or GAD_data2['20542-0.0'].iloc[i] == 1:
        num_criterias = num_criterias + 1
    if GAD_data2['20543-0.0'].iloc[i] == 2 or GAD_data2['20540-0.0'].iloc[i] == 1:
        num_criterias = num_criterias + 1
    if GAD_data2['20541-0.0'].iloc[i] == 1 or GAD_data2['20539-0.0'].iloc[i] == 3 or GAD_data2['20537-0.0'].iloc[i] == 3:
        num_criterias = num_criterias + 1
    if GAD_data2['20418-0.0'].iloc[i] >= 2:
        num_criterias = num_criterias + 1
    
    # 3 somatic symptoms out of
    num_symptoms = 0
    if GAD_data2['20426-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if GAD_data2['20423-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if GAD_data2['20429-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if GAD_data2['20419-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if GAD_data2['20422-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if GAD_data2['20417-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
    if GAD_data2['20427-0.0'].iloc[i] == 1:
        num_symptoms = num_symptoms + 1
        
    if num_symptoms >= 3:
        num_criterias = num_criterias + 1
    
    if num_criterias == 8:
        history_gad_subjs.append(GAD_data2['eid'].iloc[i])

In [67]:
controls = []
PHQ_items = ['20514-0.0','20510-0.0','20517-0.0','20519-0.0','20511-0.0','20507-0.0','20508-0.0','20518-0.0','20513-0.0']
PHQ_rows_nan = HC_data2[PHQ_items].isnull().sum(axis=1)
PHQ_data = HC_data2[PHQ_items].fillna(0)

GAD7_items = ['20506-0.0','20509-0.0','20520-0.0','20515-0.0','20516-0.0','20505-0.0','20512-0.0']
GAD7_rows_nan = HC_data2[GAD7_items].isnull().sum(axis=1)
GAD7_data = HC_data2[GAD7_items].fillna(0)

for i in range(HC_data2.shape[0]):
    subject_id = HC_data2['eid'].iloc[i]
    num_criterias = 0
    
    if HC_data2['Self_Reported_Mental_Health_Imaging_Time'].iloc[i] == 0 and \
    HC_data2['Ever_Diagnosed_Mental_Health_Problem'].iloc[i] == 'No':
        num_criterias = num_criterias + 1
    
    if (subject_id not in history_dp_subjs) and (subject_id not in history_gad_subjs) and \
    (subject_id not in l1) and (subject_id not in l2):
        num_criterias = num_criterias + 1
    
    # PHQ score <= 5:
    if PHQ_rows_nan[i] <= 2:
        PHQ_score = np.sum(PHQ_data.iloc[i,:].values) - 9
        if PHQ_score <= 5:
            num_criterias = num_criterias + 1
    
    # GAD7 score <= 5:
    if GAD7_rows_nan[i] <= 2:
        GAD7_score = np.sum(GAD7_data.iloc[i,:].values) - 9
        if GAD7_score < 5:
            num_criterias = num_criterias + 1
    
    if num_criterias == 4:
        controls.append(subject_id)

In [70]:
tmp_controls = list(set(controls) & set(subjs1))

current_status_data3 = current_status_data2.set_index('eid')
controls = []
for g in tmp_controls:
    tmp_data = current_status_data3.loc[g].values
    if tmp_data[0] == 1 and tmp_data[1] == 1 and tmp_data[2] == 1:
        controls.append(g)

In [76]:
# covariates
tdata.reset_index(inplace=True)
df = pd.DataFrame({"ID" : tdata['eid']})
df['Age'] = tdata['21003-2.0']
df['Sex'] = tdata['31-0.0']
df['IQ'] = tdata['20016-2.0']
df['EA'] = tdata['6138-2.0']
df['motion'] = tdata['25741-2.0']
df['TGMV'] = tdata['25005-2.0']
df['ICV'] = tdata['26521-2.0']
df['TCSA'] = tdata['26721-2.0'] + tdata['26822-2.0']
df['ACT'] = tdata['26755-2.0'] + tdata['26856-2.0']

In [77]:
from scipy import stats

def check_covariates_differences(l1,l2,df):
    df1 = df.set_index('ID')
    co1 = df1.loc[l1]
    co2 = df1.loc[l2]
    
    T1 = co1['Sex'].value_counts(dropna=False, sort=False)
    T2 = co2['Sex'].value_counts(dropna=False, sort=False)
    print("Sex distribution of Cases:",T1[0],T1[1])
    print("Sex distribution of Controls:",T2[0],T2[1])
    obs = np.array([[T1[0],T1[1]], [T2[0],T2[1]]])
    t1 = stats.chi2_contingency(obs)
    print("Chi-square test for sex: ", t1[:2])

    x1 = co1.iloc[:,0].values
    x2 = co2.iloc[:,0].values
    t,p = stats.ttest_ind(x1,x2)
    print('T test of age: ',t,p,np.mean(x1),np.std(x1),np.mean(x2),np.std(x2))

    x1 = co1.iloc[:,2].values
    x2 = co2.iloc[:,2].values
    t,p = stats.ttest_ind(x1,x2)
    print('T test of IQ: ',t,p,np.mean(x1),np.std(x1),np.mean(x2),np.std(x2))

    x1 = co1.iloc[:,3].values
    x2 = co2.iloc[:,3].values
    t,p = stats.ttest_ind(x1,x2)
    print('T test of EA: ',t,p,np.mean(x1),np.std(x1),np.mean(x2),np.std(x2))

In [78]:
import random

def match_two_datasets_covariates(l1,l2,df,N=None):
    df1 = df.set_index('ID')
    random.shuffle(l1)
    random.shuffle(l2)
    mdata1 = df1.loc[l1]
    mdata2 = df1.loc[l2]

    selected_l = []
    temp_l = []

    for i in range(mdata1.shape[0]):  
        for j in range(mdata2.shape[0]):
            if abs(mdata2.iloc[j,0] - mdata1.iloc[i,0]) <= 2 and \
            mdata2.iloc[j,1] == mdata1.iloc[i,1] and \
            abs(mdata2.iloc[j,2] - mdata1.iloc[i,2]) < 0.5 and \
            abs(mdata2.iloc[j,3] - mdata1.iloc[i,3]) < 0.5:
                selected_l.append(mdata2.index[j])
                temp_l.append(mdata1.index[i])
                mdata2.drop(mdata2.index[j],inplace=True)
                break

    print("the first selection step:", len(selected_l))

    rest_l = list(set(l1) - set(temp_l))
    mdata1 = df1.loc[rest_l]

    for i in range(mdata1.shape[0]):  
        for j in range(mdata2.shape[0]):
            if abs(mdata2.iloc[j,0] - mdata1.iloc[i,0]) <= 2 and \
            mdata2.iloc[j,1] == mdata1.iloc[i,1] and \
            abs(mdata2.iloc[j,2] - mdata1.iloc[i,2]) <= 1 and \
            abs(mdata2.iloc[j,3] - mdata1.iloc[i,3]) <= 3:
                selected_l.append(mdata2.index[j])
                temp_l.append(mdata1.index[i])
                mdata2.drop(mdata2.index[j],inplace=True)
                break
    print("the second selection step:", len(selected_l))

    rest_l = list(set(l1) - set(temp_l))
    mdata1 = df1.loc[rest_l]

    for i in range(mdata1.shape[0]):  
        for j in range(mdata2.shape[0]):
            if abs(mdata2.iloc[j,0] - mdata1.iloc[i,0]) <= 4 and \
            mdata2.iloc[j,1] == mdata1.iloc[i,1] and \
            abs(mdata2.iloc[j,2] - mdata1.iloc[i,2]) <= 4 and \
            abs(mdata2.iloc[j,3] - mdata1.iloc[i,3]) <= 5:
                selected_l.append(mdata2.index[j])
                temp_l.append(mdata1.index[i])
                mdata2.drop(mdata2.index[j],inplace=True)
                break

    print("the thrid selection step:", len(selected_l))
    
    if N == None:
        return selected_l
    else:
        return temp_l[:N],selected_l[:N]

In [80]:
fl3,fl4 = match_two_datasets_covariates(pure_gad_subjs,common_subjs,df,N=200)
check_covariates_differences(fl3,fl4,df)

fl1 = match_two_datasets_covariates(controls,fl3,df)
check_covariates_differences(fl3,fl1,df)

fl2 = match_two_datasets_covariates(pure_dp_subjs,fl1,df)
check_covariates_differences(fl2,fl1,df)

the first selection step: 99
the second selection step: 176
the thrid selection step: 205
Sex distribution of Cases: 112 88
Sex distribution of Controls: 112 88
Chi-square test for sex:  (0.0, 1.0)
T test of age:  -0.11434847012953843 0.9090192231195775 60.33 7.170850716616544 60.41 6.780995502136836
T test of IQ:  0.886342543299599 0.37596820282265864 6.495 1.9849370267089081 6.32 1.95386795869117
T test of EA:  0.18632087773354084 0.8522880654836269 15.435 4.903649151397356 15.345 4.7313819334312885
the first selection step: 196
the second selection step: 200
the thrid selection step: 200
Sex distribution of Cases: 112 88
Sex distribution of Controls: 112 88
Chi-square test for sex:  (0.0, 1.0)
T test of age:  0.0 1.0 60.33 7.170850716616544 60.33 7.170850716616544
T test of IQ:  0.0 1.0 6.495 1.9849370267089081 6.495 1.9849370267089081
T test of EA:  0.0 1.0 15.435 4.903649151397356 15.435 4.903649151397356
the first selection step: 144
the second selection step: 188
the thrid selec